Import packages

In [38]:
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import csv
import time
import sys
from tqdm.notebook import tqdm, trange
import random

Set URL and directory

In [39]:
base_url = 'https://debatepolitics.com/forums/2016-us-presidential-election.178/'
p_dir = 'Downloaded_pages/2016/'

pages_url = []
thread_url = {}

for i in trange(1,50):
    pages_url.append(base_url + 'page-'+str(i))

Scrape data and load onto dictonary

In [40]:
for page in tqdm(pages_url):
    page = requests.get(page)
    page_bs = BeautifulSoup(page.content, 'html.parser')
    disc = page_bs.find('div', {'class': 'structItemContainer-group js-threadList'})
    for thread in disc.find_all('div', {'class': 'structItem-title'}):
        url_val = 'https://debatepolitics.com' + thread.find('a').get('href')
        for j in range(3):
            thread_url [thread.get_text()+str(j+1)] = url_val + 'page-' + str(j+1)
            
    sleep(2)

Scrape from each thread

In [41]:
len(thread_url)

2934

In [46]:
i = 1
for title, url in tqdm(thread_url.items()):
    if requests.get(url, verify=False).status_code == 200:
        thread_html = requests.get(url, verify=False)
        with open (p_dir + str(i) + '.html', 'w+') as f:
            f.write(str(thread_html.content))
        f.close()
        i += 1
        sleep(random.uniform(5,15))
    
    else:
        sleep(50)

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [48]:
thread_data = []

for i in trange(1, len(thread_url)+1):
    with open (p_dir + str(i) + '.html', 'r') as f:
        dat = BeautifulSoup(f, 'html.parser')
        dat_com  = dat.find('div', {'class' : 'block-container lbContainer'})
        for comment in dat_com.find_all('article', {'class' : 'message message--post js-post js-inlineModContainer'}):
            comment_msg = ''.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'bbWrapper'}).get_text()))
            author = ''.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'message-userDetails'}).find('a', {'class' : 'username'}).get_text()))
            # date = comment.find('header', {'class' : 'message-attribution message-attribution--split'}).find('a').get_text('u-concealed')
            user_info = {}
            for user_extra in comment.find_all('dl', {'class' : 'pairs pairs--justified'}):
                user_info[user_extra.find('dt').get_text()] = ''.join(re.findall('[^\\\\nt]+[a-zA-Z0-9,.!@?&$%]',user_extra.find('dd').get_text()))
            # print(user_info)
            if 'Gender' in user_info and user_info['Gender'] != 'Undisclosed':
                gender = user_info['Gender']
            else:
                gender = None
            if 'Location' in user_info and user_info['Location'] != 'Undisclosed':
                location = user_info['Location']
            else:
                location = None
            if 'Political Leaning' in user_info and user_info['Political Leaning'] != 'Undisclosed':
                polLean = user_info['Political Leaning']
            else:
                polLean = None
            thread_data.append([author, comment_msg, i, gender, location, polLean])
    f.close()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [19]:
clm_names = ['Poster' , 'Content', 'Thread', 'Gender', 'Location', 'Political_leaning']

df = pd.DataFrame(thread_data, columns = clm_names)

df

,Poster,Content,Thread,Gender,Location,Political_leaning
0,bongsaway,"I think many dems, myself included, would be c...",1,Male,Flori-duh,Progressive
1,Emily L,"I honestly don't understand it, Bong. I'm a c...",1,None,Bedroom,None
2,joluot,"The GOP are scared of their base, and short te...",1,Male,None,Progressive
3,Checkerboard Strangler,"bongsaway saidI think many dems, myself includ...",1,Male,Los Angeles,None
4,Tmosler,So I did cast my first vote for presiden for T...,1,Male,None,Libertarian - Right
...,...,...,...,...,...,...
13564,d0gbreat,res borrachos saidI almost wan him to run just...,974,Male,"Denon, Texas",Very Liberal
13565,Helix,I suppose that i know who I'm voting against i...,974,Male,None,Independen
13566,lemmiwin,res borrachos saidIs that a bad thing?Click to...,974,Male,None,Libertarian - Right
13567,Fishkin,JacksinPA saidIf Trump plans to run again in 2...,974,None,None,Libertarian - Right


In [20]:
df.to_csv('2016_opinions.csv', index = False)